# Agenda

1. Review the iterator protocol
2. Generator functions
    - How to define them
    - How they're different from regular functions
    - Keeping state across invocations
    - How do they work?
3. Generator expressions (aka generator comprehensions)
    - How to define them
    - How to use them

In [1]:
# Lots of objects in Python are iterable

for one_item in 'abcde':
    print(one_item)

a
b
c
d
e


In [2]:
for one_item in [10, 20, 30, 40, 50]:
    print(one_item)

10
20
30
40
50


In [3]:
d = {'a':1, 'b':2, 'c':3}

for one_item in d:
    print(one_item)

a
b
c


In [4]:
for key, value in d.items():
    print(f'{key}: {value}')

a: 1
b: 2
c: 3
